## Question 3: Counter-Spending / Arms Race
- Aggregate to **weekly time series** within each cycle
- Compute **Arms Race Index** = min(pro_R, pro_D) / max(pro_R, pro_D) — 1.0 = perfect parity
- Apply **Granger causality tests** to assess whether spending spikes by one party predict reactive spending by the other in subsequent weeks
- Test hypothesis: massive IEs by one party trigger roughly equivalent counter-spending from the other (the strategic arms race)

In [ ]:
# Load packages + connect to database

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import os
from statsmodels.tsa.stattools import grangercausalitytests

conn = sqlite3.connect('../data/citizens_united.db')

os.makedirs('../output/figures', exist_ok=True)
os.makedirs('../output/tables',  exist_ok=True)

## 1. Load weekly partisan spending

Query `partisan_spending_weekly` — the weekly-granularity version of the core derived table.
Filter to `pro_R` and `pro_D` only. For each cycle, pivot into a DataFrame with columns `pro_R` and `pro_D`
indexed by week. Missing weeks fill as 0.

## 2. Compute Arms Race Index per cycle

Arms Race Index (ARI) = min(pro_R, pro_D) / max(pro_R, pro_D) for each week.
- 1.0 = both sides spending identically (perfect arms race)
- 0.0 = one side completely dominates
- Undefined when both sides spend $0 → exclude from summary stats

Also compute a **4-week rolling average** to smooth lumpy campaign spending patterns.
Record mean and median ARI per cycle.

## 3. Granger causality tests

Test: does pro_R spending in week *t* predict pro_D spending in week *t+k* (and vice versa)?

Use `statsmodels.tsa.stattools.grangercausalitytests` with `maxlag=4`.
Extract F-test p-values for lags 1–4 in both directions (R→D and D→R).

**Caveat:** Granger causality tests predictive precedence, not true causation.
With ~90 weekly observations per cycle, the test has reasonable power at short lags.

## 4. Print Granger summary

Print a formatted table of p-values for both directions across all cycles and lags.
Flag significant results (α=0.05 and α=0.10) with asterisks.

## 5. Figure 1 — Weekly pro-R vs. pro-D time series (2×2 grid)

One panel per cycle. Red line = pro-Republican weekly spending, blue line = pro-Democratic.
Fill the gap between lines: red-shaded where Republican leads, blue-shaded where Democratic leads.
X-axis: weeks since cycle start (Jan of odd year); Y-axis: 2024 $M.
Include OpenSecrets attribution.

## 6. Figure 2 — Arms Race Index over time (2×2 grid)

One panel per cycle showing the 4-week rolling ARI. Y-axis: 0 to 1.
Horizontal dashed lines at 0.5 and 0.8 (parity thresholds).
Green fill when ARI > 0.8 (strong arms race weeks).
Panel title includes cycle mean ARI for quick comparison.
Include OpenSecrets attribution.

## 7. Figure 3 — Granger causality heatmap

Two-panel heatmap (R→D and D→R). Rows = cycles, columns = lags 1–4.
Color scale: dark = low p-value (significant), light = high p-value.
Annotate each cell with exact p-value.
This is the main statistical figure for Q3.

## 8. Save results tables

- `output/tables/q3_granger_results.csv` — p-values for both directions, all cycles and lags
- `output/tables/q3_arms_race_index.csv` — mean/median ARI per cycle
- `output/tables/q3_weekly_spending.csv` — full weekly pro_R / pro_D data (all cycles)